<a href="https://colab.research.google.com/github/Shuaib11-Github/AI_Agents/blob/main/Building_a_Multi_Agent_System_for_Competitor_Analysis_and_Real_Time_Strategy_Adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Multi-Agent System for Competitor Analysis and Real-Time Strategy Adjustment

This tutorial guides you through building a multi-agent system for **competitor analysis and real-time strategy adjustment** using **OpenAI's APIs**, **LangChain**, and other modern libraries. This system will:
- Monitor competitor activities.
- Analyze market data.
- Generate strategic recommendations for real-time adjustments.

The tutorial includes a step-by-step walkthrough, a fully functional Google Colab notebook, and integration with **Gradio** for a user-friendly interface.

---

## **Key Libraries Used**
We leverage the following Python libraries:
- **OpenAI**: For language model capabilities.
- **LangChain**: To manage agents, tools, and memory.
- **ChromaDB**: To store and retrieve competitor data.
- **Gradio**: To create a user interface.
- **BeautifulSoup**: For web scraping competitor websites.
- **Pandas** and **NumPy**: For data manipulation.

---

## **1. Environment Setup**
Ensure all dependencies are installed. Use the following command to install the required libraries:

In [ ]:
!pip install openai gradio chromadb langchain pandas numpy requests beautifulsoup4 langchain_community tiktoken python-dotenv
# !pip install openai gradio chromadb langchain pandas numpy requests beautifulsoup4 langchain_community tiktoken python-dotenv git+https://github.com/openai/swarm.git

## **2. Competitor Analysis Agent**
The **CompetitorAnalysisAgent** is the core of our system. It processes competitor data and generates insights using OpenAI’s language model.

In [ ]:
import os
import openai
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory
import chromadb
from typing import Dict
from google.colab import userdata
# Set OpenAI API key
openai.api_key = userdata.get('openai')
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

class CompetitorAnalysisAgent:
    def __init__(self, competitor_name: str):
        self.competitor_name = competitor_name
        self.llm = OpenAI(temperature=0.7)
        self.memory = ConversationBufferMemory()
        self.chroma_client = chromadb.Client()
        self.collection = self.chroma_client.create_collection(name=f"competitor_{competitor_name}")

    def analyze_competitor_data(self, data: Dict) -> str:
        """
        Analyzes competitor data using LLM.
        """
        template = """
        Analyze the following competitor data and provide strategic insights:
        Competitor: {competitor}
        Data: {data}
        Previous context: {history}

        Provide insights on:
        1. Market positioning
        2. Pricing strategy
        3. Product features
        4. Marketing approach
        5. Recommended counter-strategies
        """
        prompt = PromptTemplate(
            input_variables=["competitor", "data", "history"],
            template=template
        )
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(
            competitor=self.competitor_name,
            data=data,
            history=self.memory.buffer
        )



## **3. Competitor Data Scraper**
The **CompetitorDataScraper** collects real-time data from competitor websites using web scraping.

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

class CompetitorDataScraper:
    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0'}

    def scrape_competitor_website(self, url: str) -> Dict:
        """
        Scrapes competitor website for relevant information.
        """
        try:
            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            data = {
                'title': soup.title.string if soup.title else '',
                'timestamp': datetime.now().isoformat()
                # Add custom logic for extracting products or pricing here.
            }
            return data
        except Exception as e:
            print(f"Error scraping website: {e}")
            return {}

## **4. Strategy Adjustment System**
This module generates actionable strategies based on competitor analysis.

In [ ]:
class StrategyAdjustmentSystem:
    def __init__(self):
        self.llm = OpenAI(temperature=0.5)
        self.memory = ConversationBufferMemory()

    def generate_strategy_recommendations(self, competitor_analysis: str) -> Dict:
        """
        Generates strategic recommendations from analysis.
        """
        template = """
        Based on the following competitor analysis, provide actionable strategies:
        Analysis: {analysis}

        Focus on:
        1. Pricing
        2. Product improvements
        3. Marketing strategies
        4. Competitive advantages

        Return the recommendations as a structured action plan.
        """
        prompt = PromptTemplate(
            input_variables=["analysis"],
            template=template
        )
        chain = LLMChain(llm=self.llm, prompt=prompt)
        return chain.run(analysis=competitor_analysis)

## **5. Competitor Monitoring System**
This system combines scraping, analysis, and strategy generation for monitoring competitors.

In [ ]:
class CompetitorMonitoringSystem:
    def __init__(self):
        self.competitors = {}
        self.scraper = CompetitorDataScraper()
        self.strategy_system = StrategyAdjustmentSystem()
        self.chroma_client = chromadb.Client()

    def add_competitor(self, name: str, url: str):
        """
        Adds a competitor to the monitoring system.
        """
        self.competitors[name] = {
            'url': url,
            'agent': CompetitorAnalysisAgent(name)
        }

    def monitor_competitors(self):
        """
        Monitors all competitors and generates insights.
        """
        results = []
        for name, data in self.competitors.items():
            competitor_data = self.scraper.scrape_competitor_website(data['url'])
            data['agent'].analyze_competitor_data(competitor_data)
            analysis = data['agent'].analyze_competitor_data(competitor_data)
            recommendations = self.strategy_system.generate_strategy_recommendations(analysis)
            results.append({'competitor': name, 'analysis': analysis, 'recommendations': recommendations})
        return results

## **6. User Interface with Gradio**
Finally, we create an intuitive **Gradio** interface for adding competitors, running analyses, and viewing historical data.

In [ ]:
import gradio as gr

class CompetitorAnalysisUI:
    def __init__(self):
        self.monitoring_system = CompetitorMonitoringSystem()

    def format_results(self, results):
        """
        Formats the analysis results into a styled HTML for professional display.
        """
        html = "<h2>Analysis Results</h2>"
        for idx, result in enumerate(results):
            competitor = result["competitor"]
            analysis = result["analysis"]
            recommendations = result["recommendations"]
            html += f"""
            <div style='margin-bottom: 20px; padding: 15px; border: 1px solid #ddd; border-radius: 8px;'>
                <h3 style='color: #2c3e50;'>Competitor {idx + 1}: {competitor}</h3>
                <strong style='color: #16a085;'>Analysis:</strong>
                <p style='font-size: 14px; line-height: 1.6;'>{analysis}</p>
                <strong style='color: #c0392b;'>Recommendations:</strong>
                <ul style='font-size: 14px; line-height: 1.6;'>
                    {"".join(f"<li>{rec.strip()}</li>" for rec in recommendations.split("-"))}
                </ul>
            </div>
            """
        return html

    def setup_interface(self):
        """
        Creates a polished Gradio interface for the competitor analysis system.
        """
        def add_competitor(name, url):
            self.monitoring_system.add_competitor(name, url)
            return f"Competitor '{name}' successfully added!"

        def run_analysis():
            results = self.monitoring_system.monitor_competitors()
            formatted_results = self.format_results(results)
            return formatted_results

        # Gradio Interface
        with gr.Blocks(theme=gr.themes.Soft()) as interface:
            # Landing Page Header
            with gr.Row():
                gr.Markdown(
                    """
                    # 🌟 **Competitor Analysis System**
                    Welcome to the Competitor Analysis System! Use this tool to monitor and analyze your competitors effortlessly.

                    ### Features:
                    - Add competitor information (name and website).
                    - Run comprehensive competitor analysis with a single click.

                    Navigate using the tabs below:
                    """
                )

            # Tabs for functionality
            with gr.Tab("🔗 Add Competitor"):
                gr.Markdown("### Add Competitor Details")
                name_input = gr.Textbox(label="Competitor Name", placeholder="Enter competitor's name")
                url_input = gr.Textbox(label="Website URL", placeholder="Enter competitor's website URL")
                add_button = gr.Button("➕ Add Competitor", elem_id="add-btn")
                add_output = gr.Textbox(label="Message", interactive=False)
                add_button.click(add_competitor, inputs=[name_input, url_input], outputs=add_output)

            with gr.Tab("📊 Run Analysis"):
                gr.Markdown("### Perform Competitor Analysis")
                run_button = gr.Button("🚀 Run Analysis", elem_id="run-btn")
                analysis_output = gr.HTML(label="Analysis Results")
                run_button.click(run_analysis, outputs=analysis_output)

            # Footer
            gr.Markdown(
                """
                ---
                *Powered by AI for Competitive Intelligence.*
                **Contact:** [support@example.com](mailto:support@example.com)
                """
            )

        return interface

## **7. Main Application**
Launch the system using the following script:

In [ ]:
def main():
    ui = CompetitorAnalysisUI()
    interface = ui.setup_interface()
    interface.launch(share=True, debug=True)

if __name__ == "__main__":
    main()

<ipython-input-4-9bedf66741f5>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  self.llm = OpenAI(temperature=0.5)
<ipython-input-4-9bedf66741f5>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aface567ce800b1450.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-2-05a4e041a503>:41: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=prompt)
<ipython-input-2-05a4e041a503>:42: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://aface567ce800b1450.gradio.live


## **Conclusion**
This multi-agent system provides:
1. **Real-time competitor monitoring** through scraping.
2. **Deep insights** using OpenAI’s LLMs.
3. **Actionable strategies** for business decisions.

You can customize and expand this system by:
- Adding advanced scraping rules.
- Storing more data in **ChromaDB**.
- Enhancing Gradio UI for visualization.